# Exercise of Text Feature Extraction

**Nguyen Luu Phuong Ngoc Lam - 240108**

In [11]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer, HashingVectorizer
import json
import re
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        print(file_path)


/kaggle/input/vietnamese-online-news-dataset/news_dataset.json


In [6]:
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.json_normalize(data)

# View dataset structure
print(df.head())

       id             author  \
0  218270                      
1  218269      (Nguồn: Sina)   
2  218268          Hồ Sỹ Anh   
3  218267           Ngọc Ánh   
4  218266  HẢI YẾN - MINH LÝ   

                                             content  picture_count  \
0  Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...              3   
1  Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...              1   
2  Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...              3   
3  Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...              1   
4  Vụ tai nạn giao thông liên hoàn trên phố đi bộ...             12   

   processed        source                                              title  \
0          0     docbao.vn  Tên cướp tiệm vàng tại Huế là đại uý công an, ...   
1          0        vtc.vn        Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G   
2          0  thanhnien.vn  Địa phương nào đứng đầu cả nước tổng điểm 3 mô...   
3          0     vnexpress  Người chết trong mưa lũ 'ngh

In [7]:
# Text cleaning function
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

df['clean_content'] = df['content'].apply(clean_text)

# Split dataset into train and test
X_train, X_test = train_test_split(df['clean_content'], test_size=0.2, random_state=42)


### Bag of Words (BoW)

In [8]:
# Initialize CountVectorizer (BoW)
bow_vectorizer = CountVectorizer()
X_bow_train = bow_vectorizer.fit_transform(X_train)
X_bow_test = bow_vectorizer.transform(X_test)

# Show feature names (top 10 words)
print(bow_vectorizer.get_feature_names_out()[:10])

['00' '000' '0000' '000000' '0000008599' '000001' '0000012000016' '000002'
 '000005' '000009']


### TF-IDF

In [12]:
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_tfidf_train = tfidf_vectorizer.fit_transform(X_train)
X_tfidf_test = tfidf_vectorizer.transform(X_test)

# Show TF-IDF feature names (top 10 words)
print(tfidf_vectorizer.get_feature_names_out()[:10])

['00' '000' '0000' '000000' '0000008599' '000001' '0000012000016' '000002'
 '000005' '000009']


### Unigram

In [13]:
# Unigram model using CountVectorizer
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))  # Only unigrams
X_unigram_train = unigram_vectorizer.fit_transform(X_train)
X_unigram_test = unigram_vectorizer.transform(X_test)

# Show unigram features (top 10)
print(unigram_vectorizer.get_feature_names_out()[:10])


['00' '000' '0000' '000000' '0000008599' '000001' '0000012000016' '000002'
 '000005' '000009']


### Bigram

In [14]:
# Bigram model using CountVectorizer
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))  # Only bigrams
X_bigram_train = bigram_vectorizer.fit_transform(X_train)
X_bigram_test = bigram_vectorizer.transform(X_test)

# Show bigram features (top 10)
print(bigram_vectorizer.get_feature_names_out()[:10])


['00 10' '00 100' '00 17' '00 186' '00 19' '00 2323' '00 249' '00 30'
 '00 35' '00 38']


### Compare top features across the four methods

In [15]:
# Function to display top features for comparison
def display_top_features(vectorizer, X, top_n=10):
    features = vectorizer.get_feature_names_out()
    sums = X.sum(axis=0)
    data = []
    for col, term in enumerate(features):
        data.append((term, sums[0, col]))
    ranking = pd.DataFrame(data, columns=['term', 'occurrences'])
    top_features = ranking.sort_values('occurrences', ascending=False).head(top_n)
    print(top_features)

print("Top 10 Bag of Words (BoW) Features:")
display_top_features(bow_vectorizer, X_bow_train)

print("\nTop 10 TF-IDF Features:")
display_top_features(tfidf_vectorizer, X_tfidf_train)

print("\nTop 10 Unigram Features:")
display_top_features(unigram_vectorizer, X_unigram_train)

print("\nTop 10 Bigram Features:")
display_top_features(bigram_vectorizer, X_bigram_train)

Top 10 Bag of Words (BoW) Features:
         term  occurrences
192598     và       972473
72152     của       848105
71665      có       802927
71551     các       697077
138662     là       635370
184681  trong       626262
193030    với       573399
66780     cho       543213
200495   được       541943
131103  không       508907

Top 10 TF-IDF Features:
         term  occurrences
192598     và  7270.929590
72152     của  6515.945564
71665      có  5972.231942
71551     các  5718.223385
138662     là  4921.186594
184681  trong  4893.457394
193030    với  4504.914988
150413  người  4454.482516
66780     cho  4401.591995
200495   được  4362.843039

Top 10 Unigram Features:
         term  occurrences
192598     và       972473
72152     của       848105
71665      có       802927
71551     các       697077
138662     là       635370
184681  trong       626262
193030    với       573399
66780     cho       543213
200495   được       541943
131103  không       508907

Top 10 Bigram Feature